In [ ]:
%pip install langchain_community langchain_text_splitters langchain_google_genai langchain_classic faiss-cpu

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.llms import VertexAI
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_google_genai import ChatGoogleGenerativeAI
import getpass
import os
from IPython.display import display, Markdown

In [ ]:
# Load Document (here from the web: FastAPI Official Website)
url = "https://fastapi.tiangolo.com/"
loader = WebBaseLoader(url)
raw_documents = loader.load()

In [ ]:
# Chunk the document text
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(raw_documents)

In [ ]:
# Get the API Key and Generate Text Vector Embeddings
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")
    
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")


In [ ]:
# Create a vector store and enable memory
vector_store = FAISS.from_documents(documents,embeddings)
memory = ConversationBufferMemory(memory_key = "chat_history", return_messages = True)

In [ ]:
# Call/instantiate the LLM and the chat
qa = ConversationalRetrievalChain.from_llm(ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key = os.environ["GOOGLE_API_KEY"],
    project= os.environ["GOOGLE_PROJECT_ID"],
    vertexai = False,
    temperature= 0),
    vector_store.as_retriever(),
    memory = memory)

In [ ]:
# Ask your contextual question
query = "How to set up FastAPI?"
result = qa.invoke({"question": query})


In [ ]:
# Eureka! (No these comments are not AI but written by me and me alone).
display(Markdown(result["answer"]))

In [ ]:
query2 = "I am getting an erro rwhen trying to run fastapi dev main.py. Any ideas?"
result2 = qa.invoke({"question": query2})

In [ ]:
display(Markdown(result2["answer"]))

In [ ]:
query3 = "How do you know about these common reasons"
result3 = qa.invoke({"question": query3})

In [ ]:
display(Markdown(result3["answer"]))

In [ ]:
query4 = "Describe how FastAPI works like you are explaining it to a chef"
result4 = qa.invoke({"question": query4})

In [ ]:
display(Markdown(result4["answer"]))

In [ ]:
query5 = "How can I deploy my app?"
result5 = qa.invoke({"question": query5})

In [ ]:
display(Markdown(result5["answer"]))